<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial8_About_Splitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Tutorial Part8: 分割方法について
DeepChemが提供する分割方法を紹介

#### Setup

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  20529      0 --:--:-- --:--:-- --:--:-- 20650


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added omnia to channels
added conda-forge to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [3]:
!pip install --pre deepchem
import deepchem as dc
deepchem.__version__

'2.5.0.dev'

#### Splitters
DeepChemでは、Splitterオブジェクトによってデータを複数のデータセットに分割する方法が定義されている。データに適した方法を選択することは非常に重要。そうでないと、学習後のモデルが実際よりも上手く予測できているように見えてしまうことがあるため。

一般的な医薬品開発パイプラインを考えてみる。
何千もの分子をスクリーニングして、興味のあるターゲットに結合するかどうかを確認することから始まる。うまくいきそうなものを見つけたら、その上で何千ものバリエーションをテストして、より強く結合するものを探して最適化していく。その後、動物実験をして許容できない毒性があることがわかった場合には、問題を解決するために、より多くのバリエーションを試すことになる。

これは化学のデータセットに対して重要な結果をもたらす。
化学のデータセットには多くの場合、互いに非常に似た分子が含まれている。これを一般的な方法で学習データとテストデータに分割すると学習セットには、たとえそれらが完全に同一ではなくても、テストセットの分子と非常に類似している分子が多く含まれる。その結果、テストセットでうまくいくかもしれないが、学習データとあまり似ていないデータではひどく失敗してしまう。

- RandomSplitter
    - ランダムな方法で、学習データ・検証データ・テストデータに分割する方法。
    - 非常に似た分子が含まれているデータセットの場合は、あまり良い分割とは言えない。

- RandomStratifiedSplitter(ランダム層化スプリッター)
    - 陽性、陰性のデータがアンバランスなデータセットに対して使用する分割方法。
    - 陽性、陰性のデータを均等に分割する。

- ScaffoldSplitter
    - 多くの分子が非常に似ているという問題に対処するための分割方法。
    - 各分子の骨格となる構造を特定し、同じ構造を有するすべての分子が同じデータセットに含まれるようにする。2つの分子が異なる骨格を有していても、ほかの点で非常に似ていることがあるため、完全な対応策でない。ただ、ランダムな分割よりも大幅に改善される。

- ButinaSplitter
    - 類似分子の問題を解決する別の分割方法。
    - Fingerprintに基づいて分子をクラスタリングし、類似したFingerprintを持つものが同じデータセットに含まれるようにする。
    - 分割アルゴリズムの所要時間は、分子数の2乗に比例するため、小規模～中規模データセットに有用。

- SpecifiedSplitter
    - ユーザーが自身で分割する。
    - データに対しての適した分割方法を事前に基地である場合に便利。
    - 例えば、時間的分割がある。新しい分子を継続的に生成してテストしている研究プロジェクトを考える。データが増えてくると、確実に増えてきたデータセットで定期的にモデルを再訓練し、そのモデルを使って、まだテストしていないほかの分子の結果を予測する。

In [14]:
# splitterごとの性能評価
splitters = ['random', 'scaffold', 'butina']
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

for splitter in splitters:
    tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP', splitter=splitter)
    train, valid, test = datasets
    model = dc.models.MultitaskClassifier(n_tasks=len(tasks), n_features=1024, layer_sizes=[1000])
    model.fit(train, nb_epoch=10)
    print('splitter: ', splitter)
    print('training set score: {}'.format(model.evaluate(train, [metric], transformers)))
    print('test set score: {}'.format(model.evaluate(test, [metric], transformers)))
    print('*'*20)

splitter:  random
training set score: {'roc_auc_score': 0.9549845702836771}
test set score: {'roc_auc_score': 0.7646855918127599}
********************
splitter:  scaffold
training set score: {'roc_auc_score': 0.9579291201300105}
test set score: {'roc_auc_score': 0.6785832816646115}
********************
splitter:  butina
training set score: {'roc_auc_score': 0.9586535060774185}
test set score: {'roc_auc_score': 0.6023001404023747}
********************


randomが精度が高いが、優れているわけではない。
テスト集合には、訓練集合と非常によく似た分子が多く含まれているため、真の意味での独立性はない。